In [20]:
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta

# Load your dataset
df = pd.read_csv('/Users/caitlyndinh/ai-academy/FinalProject/merged_with_naics.csv', parse_dates=['ApprovalDate', 'AsOfDate'], low_memory=False)

# Step 1: Filter only current loans
df['LoanStatus'] = df['LoanStatus'].astype(str).str.strip().str.upper()
df = df[df['LoanStatus'] == 'EXEMPT'].copy()
total_loans = len(df)

# Step 2: Identify rows with missing values in required columns
required_cols = ['GrossApproval', 'SBAGuaranteedApproval', 'ApprovalDate', 'AsOfDate', 'TermInMonths', 'InitialInterestRate']
missing_counts = df[required_cols].isna().sum()
num_missing_loans = df[required_cols].isnull().any(axis=1).sum()

# Step 3: Drop rows with missing critical fields
df = df.dropna(subset=required_cols).copy()
cleaned_loans = len(df)

# Step 4: Calculate months elapsed using relativedelta
def calculate_months_elapsed(row):
    if pd.isnull(row['AsOfDate']) or pd.isnull(row['ApprovalDate']):
        return np.nan
    delta = relativedelta(row['AsOfDate'], row['ApprovalDate'])
    return delta.years * 12 + delta.months

df['MonthsElapsed'] = df.apply(calculate_months_elapsed, axis=1)
df['MonthsElapsed'] = df['MonthsElapsed'].clip(lower=0)

# Step 5: Calculate monthly interest rate
df['MonthlyRate'] = df['InitialInterestRate'] / 12 / 100

# Step 6: Amortization functions
def compute_monthly_payment(P, r, n):
    if r == 0 or n == 0:
        return 0
    return P * r * (1 + r)**n / ((1 + r)**n - 1)

def remaining_balance(P, r, n, t):
    if n == 0:
        return 0  # No term — can't calculate amortization
    if r == 0:
        return max(P * (1 - t / n), 0)
    try:
        return P * ((1 + r) ** n - (1 + r) ** t) / ((1 + r) ** n - 1)
    except ZeroDivisionError:
        return 0

# Step 7: Apply payment and balance calculations
df['MonthlyPayment'] = df.apply(lambda row: compute_monthly_payment(
    row['GrossApproval'], row['MonthlyRate'], row['TermInMonths']
), axis=1)

df['RemainingBalance'] = df.apply(lambda row: remaining_balance(
    row['GrossApproval'], row['MonthlyRate'], row['TermInMonths'], row['MonthsElapsed']
), axis=1)

# Step 8: Guaranteed portion of the remaining balance
df['RemainingGuaranteedBalance'] = df['SBAGuaranteedApproval'] * (df['RemainingBalance'] / df['GrossApproval'])

# Step 9: Apply FY2025 servicing fee rates
def servicing_fee_rate(gross):
    if gross <= 500_000:
        return 0.00
    elif gross <= 1_000_000:
        return 0.0017
    else:
        return 0.0055

df['ServicingFeeRate'] = df['GrossApproval'].apply(servicing_fee_rate)
df['AnnualFeeIncome'] = df['RemainingGuaranteedBalance'] * df['ServicingFeeRate']

# Step 10: Final total and export
total_fee_income = df['AnnualFeeIncome'].sum()
df.to_csv("projected_fee_income_cleaned.csv", index=False)

# Step 11: Print summary
print("\n SBA 7(a) FY2025 Servicing Fee Projection")
print("--------------------------------------------")
print(f"Total loans before cleaning:     {total_loans:,}")
print(f"Loans excluded (missing values): {num_missing_loans:,}")
print(f"Remaining loans used:            {cleaned_loans:,}")
print("\nMissing value counts by column:")
print(missing_counts)
print(f"\n Total FY2025 Projected Servicing Fee Income: ${total_fee_income:,.2f}")



 SBA 7(a) FY2025 Servicing Fee Projection
--------------------------------------------
Total loans before cleaning:     280,629
Loans excluded (missing values): 4,155
Remaining loans used:            276,474

Missing value counts by column:
GrossApproval               0
SBAGuaranteedApproval       0
ApprovalDate                0
AsOfDate                    0
TermInMonths                0
InitialInterestRate      4155
dtype: int64

 Total FY2025 Projected Servicing Fee Income: $343,548,535.56


Replace missing interest rate with average

In [ ]:
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta

# Load your dataset
df = pd.read_csv('/Users/caitlyndinh/ai-academy/FinalProject/merged_with_naics.csv',
                 parse_dates=['ApprovalDate', 'AsOfDate'], low_memory=False)

# Step 1: Filter only current loans
df['LoanStatus'] = df['LoanStatus'].astype(str).str.strip().str.upper()
df = df[df['LoanStatus'] == 'EXEMPT'].copy()
total_loans = len(df)

# Step 2: Identify rows with missing values in required columns
required_cols = ['GrossApproval', 'SBAGuaranteedApproval', 'ApprovalDate', 'AsOfDate', 'TermInMonths', 'InitialInterestRate']
missing_counts = df[required_cols].isna().sum()
num_missing_loans = df[required_cols].isnull().any(axis=1).sum()

# Step 2.5: Fill missing InitialInterestRate with FY average, fallback to overall average
def get_fiscal_year(date):
    return date.year + 1 if date.month >= 10 else date.year

df['FiscalYear'] = df['ApprovalDate'].apply(get_fiscal_year)

# Count missing before filling
initial_missing_rate_count = df['InitialInterestRate'].isna().sum()

# Compute fiscal year averages and fallback
fy_avg_interest = df.groupby('FiscalYear')['InitialInterestRate'].mean()
overall_avg_rate = df['InitialInterestRate'].mean()

# Track fill types
fy_fill_count = 0
fallback_fill_count = 0

# Custom fill function with tracking
def fill_missing_rate(row):
    global fy_fill_count, fallback_fill_count
    if pd.isnull(row['InitialInterestRate']):
        fy_avg = fy_avg_interest.get(row['FiscalYear'], np.nan)
        if not pd.isnull(fy_avg):
            fy_fill_count += 1
            return fy_avg
        else:
            fallback_fill_count += 1
            return overall_avg_rate
    return row['InitialInterestRate']

# Apply fill function
df['InitialInterestRate'] = df.apply(fill_missing_rate, axis=1)

# Count after filling
final_missing_rate_count = df['InitialInterestRate'].isna().sum()
filled_total = initial_missing_rate_count - final_missing_rate_count

# Step 3: Drop rows with missing critical fields
df = df.dropna(subset=required_cols).copy()
cleaned_loans = len(df)

# Step 4: Calculate months elapsed using relativedelta
def calculate_months_elapsed(row):
    if pd.isnull(row['AsOfDate']) or pd.isnull(row['ApprovalDate']):
        return np.nan
    delta = relativedelta(row['AsOfDate'], row['ApprovalDate'])
    return delta.years * 12 + delta.months

df['MonthsElapsed'] = df.apply(calculate_months_elapsed, axis=1)
df['MonthsElapsed'] = df['MonthsElapsed'].clip(lower=0)

# Step 5: Calculate monthly interest rate
df['MonthlyRate'] = df['InitialInterestRate'] / 12 / 100

# Step 6: Amortization functions
def compute_monthly_payment(P, r, n):
    if r == 0 or n == 0:
        return 0
    return P * r * (1 + r)**n / ((1 + r)**n - 1)

def remaining_balance(P, r, n, t):
    if n == 0:
        return 0  # No term — can't calculate amortization
    if r == 0:
        return max(P * (1 - t / n), 0)
    try:
        return P * ((1 + r) ** n - (1 + r) ** t) / ((1 + r) ** n - 1)
    except ZeroDivisionError:
        return 0

# Step 7: Apply payment and balance calculations
df['MonthlyPayment'] = df.apply(lambda row: compute_monthly_payment(
    row['GrossApproval'], row['MonthlyRate'], row['TermInMonths']
), axis=1)

df['RemainingBalance'] = df.apply(lambda row: remaining_balance(
    row['GrossApproval'], row['MonthlyRate'], row['TermInMonths'], row['MonthsElapsed']
), axis=1)

# Step 8: Guaranteed portion of the remaining balance
df['RemainingGuaranteedBalance'] = df['SBAGuaranteedApproval'] * (df['RemainingBalance'] / df['GrossApproval'])

# Step 9: Apply FY2025 servicing fee rates
def servicing_fee_rate(gross):
    if gross <= 500_000:
        return 0.00
    elif gross <= 1_000_000:
        return 0.0017
    else:
        return 0.0055

df['ServicingFeeRate'] = df['GrossApproval'].apply(servicing_fee_rate)
df['AnnualFeeIncome'] = df['RemainingGuaranteedBalance'] * df['ServicingFeeRate']

# Step 10: Final total and export
total_fee_income = df['AnnualFeeIncome'].sum()
df.to_csv("projected_fee_income_cleaned.csv", index=False)

# Step 11: Print summary
print("\n SBA 7(a) FY2025 Servicing Fee Projection")
print("--------------------------------------------")
print(f"Total loans before cleaning:     {total_loans:,}")
print(f"Loans excluded (missing values): {num_missing_loans:,}")
print(f"Remaining loans used:            {cleaned_loans:,}")
print("\nMissing value counts by column:")
print(missing_counts)
print(f"\n Total FY2025 Projected Servicing Fee Income: ${total_fee_income:,.2f}")

# Step 12: Interest Rate Fill Summary
print(f"\n Missing Interest Rate Handling")
print("--------------------------------")
print(f"Initially missing:         {initial_missing_rate_count:,}")
print(f"Filled using FY average:   {fy_fill_count:,}")
print(f"Filled using fallback avg: {fallback_fill_count:,}")
print(f"Still missing after fill:  {final_missing_rate_count:,}")



📊 SBA 7(a) FY2025 Servicing Fee Projection
--------------------------------------------
Total loans before cleaning:     280,629
Loans excluded (missing values): 4,155
Remaining loans used:            280,629

Missing value counts by column:
GrossApproval               0
SBAGuaranteedApproval       0
ApprovalDate                0
AsOfDate                    0
TermInMonths                0
InitialInterestRate      4155
dtype: int64

💰 Total FY2025 Projected Servicing Fee Income: $341,386,601.50

🔧 Missing Interest Rate Handling
--------------------------------
Initially missing:         4,155
Filled using FY average:   87
Filled using fallback avg: 4,068
Still missing after fill:  0
